In [1]:
import math
import tqdm
import uproot
from pathlib import Path
import awkward as ak
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['font.size'] = 12

## Getting the file and tree

In [2]:
fname = Path("/Users/alexandertuna/Downloads/cms/lst_playing/data/trackingNtuple.2025_03_21_11h59m00s.10muon_0p5gev_1p5gev.root")
if not fname.exists():
    raise Exception("shit")

In [3]:
file = uproot.open(f"{fname}")
print(file.keys())

['trackingNtuple;1', 'trackingNtuple/tree;14', 'trackingNtuple/tree;13']


In [4]:
tree = uproot.open(f"{fname}:trackingNtuple/tree")
print(tree)

<TTree 'tree' (360 branches) at 0x0001158c86e0>


In [5]:
def get_prefixes(col):
    return sorted(list(set([obj.split("_")[0] for obj in col])))
print(get_prefixes(tree.keys()))

['bsp', 'event', 'inv', 'lumi', 'ph2', 'pix', 'run', 'see', 'sim', 'simhit', 'simpv', 'simvtx', 'tcand', 'trk', 'vtx']


## Getting branches into a data array

In [6]:
print(tree.keys())

['event', 'lumi', 'run', 'trk_px', 'trk_py', 'trk_pz', 'trk_pt', 'trk_inner_px', 'trk_inner_py', 'trk_inner_pz', 'trk_inner_pt', 'trk_outer_px', 'trk_outer_py', 'trk_outer_pz', 'trk_outer_pt', 'trk_eta', 'trk_lambda', 'trk_cotTheta', 'trk_phi', 'trk_dxy', 'trk_dz', 'trk_dxyPV', 'trk_dzPV', 'trk_dxyClosestPV', 'trk_dzClosestPV', 'trk_ptErr', 'trk_etaErr', 'trk_lambdaErr', 'trk_phiErr', 'trk_dxyErr', 'trk_dzErr', 'trk_refpoint_x', 'trk_refpoint_y', 'trk_refpoint_z', 'trk_nChi2', 'trk_nChi2_1Dmod', 'trk_ndof', 'trk_mva', 'trk_qualityMask', 'trk_q', 'trk_nValid', 'trk_nLost', 'trk_nInactive', 'trk_nPixel', 'trk_nStrip', 'trk_nOuterLost', 'trk_nInnerLost', 'trk_nOuterInactive', 'trk_nInnerInactive', 'trk_nPixelLay', 'trk_nStripLay', 'trk_n3DLay', 'trk_nLostLay', 'trk_nCluster', 'trk_algo', 'trk_originalAlgo', 'trk_algoMask', 'trk_stopReason', 'trk_isHP', 'trk_seedIdx', 'trk_vtxIdx', 'trk_simTrkIdx', 'trk_simTrkShareFrac', 'trk_simTrkNChi2', 'trk_bestSimTrkIdx', 'trk_bestFromFirstHitSimTrkId

In [7]:
data = tree.arrays([
    'ph2_x', 'ph2_y', 'ph2_z',
    'ph2_order', 'ph2_side', 'ph2_layer',
    'ph2_rod', 'ph2_module', 'ph2_isStack',
    'ph2_isUpper', 'ph2_isLower', 'ph2_detId',
    'ph2_subdet', 'ph2_moduleType', 'ph2_ring',

    
    'simhit_x', 'simhit_y', 'simhit_z',
    'simhit_px', 'simhit_py', 'simhit_pz',
    'simhit_tof', 'simhit_particle', 'simhit_simTrkIdx', 
    'simhit_order', 'simhit_side', 'simhit_layer',
    'simhit_ring', 'simhit_subdet', 'simhit_moduleType',
    'simhit_rod', 'simhit_module', 'simhit_isStack',
    'simhit_ladder', 'simhit_blade', 'simhit_panel',
    'simhit_isUpper', 'simhit_isLower', 'simhit_detId',
])

## Compare equivalence of detid with individual variables

In [8]:
reco = "ph2"
detids      = ak.flatten(data[f"{reco}_detId"]).to_numpy()
orders      = ak.flatten(data[f"{reco}_order"]).to_numpy()
sides       = ak.flatten(data[f"{reco}_side"]).to_numpy()
layers      = ak.flatten(data[f"{reco}_layer"]).to_numpy()
rods        = ak.flatten(data[f"{reco}_rod"]).to_numpy()
modules     = ak.flatten(data[f"{reco}_module"]).to_numpy()
rings       = ak.flatten(data[f"{reco}_ring"]).to_numpy()
isUppers    = ak.flatten(data[f"{reco}_isUpper"]).to_numpy()
isLowers    = ak.flatten(data[f"{reco}_isLower"]).to_numpy()
isStacks    = ak.flatten(data[f"{reco}_isStack"]).to_numpy()
moduleTypes = ak.flatten(data[f"{reco}_moduleType"]).to_numpy()
subdets     = ak.flatten(data[f"{reco}_subdet"]).to_numpy()


for hit in tqdm.tqdm(range(len(detids))):
    mask_detid = detids[hit] == detids
    mask_other = \
      (orders[hit]   == orders) & \
      (sides[hit]    == sides) & \
      (layers[hit]   == layers) & \
      (rods[hit]     == rods) & \
      (modules[hit]  == modules) & \
      (rings[hit]    == rings) & \
      (isUppers[hit] == isUppers) & \
      (isLowers[hit] == isLowers) & \
      (isStacks[hit] == isStacks)

    if not np.array_equal(mask_detid, mask_other):
        print(f"Not equal! hit {hit}")
        break

else:
    print("All equal!")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1172838/1172838 [15:34<00:00, 1254.88it/s]

All equal!
